# MODEL TEST

Let's get started!

In [13]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.InteractiveSession(config=config)

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
import time
from scipy.stats import norm

%matplotlib inline
plt.style.use('ggplot')

C:\Users\ijan\anaconda3\envs\py36-tf114\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [14]:
SSD_GRAPH_FILE = 'traffic_light_inference_graph/frozen_inference_graph.pb'

Below are utility functions. The main purpose of these is to draw the bounding boxes back onto the original image.

In [15]:
#Colors (one for each class)
cmap = ImageColor.colormap
COLOR_LIST = sorted([c for c in cmap.keys()])
idx=0
#Utility funcs


def filter_boxes(min_score, boxes, scores, classes):
    """Return boxes with a confidence >= `min_score`"""
    n = len(classes)
    idxs = []
    for i in range(n):
        if scores[i] >= min_score:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    return filtered_boxes, filtered_scores, filtered_classes

def to_image_coords(boxes, height, width):
    """
    The original box coordinate output is normalized, i.e [0, 1].
    
    This converts it back to the original coordinate based on the image
    size.
    """
    box_coords = np.zeros_like(boxes)
    box_coords[:, 0] = boxes[:, 0] * height
    box_coords[:, 1] = boxes[:, 1] * width
    box_coords[:, 2] = boxes[:, 2] * height
    box_coords[:, 3] = boxes[:, 3] * width
    
    return box_coords

def draw_boxes(image, boxes, classes, thickness=4):
    """Draw bounding boxes on the image"""
    draw = ImageDraw.Draw(image)
    for i in range(len(boxes)):
        bot, left, top, right = boxes[i, ...]
        class_id = int(classes[i])
        color = COLOR_LIST[class_id]
        draw.line([(left, top), (left, bot), (right, bot), (right, top), (left, top)], width=thickness, fill=color)
        
def load_graph(graph_file):
    
    """Loads a frozen inference graph"""
    """graph_file: the path of the file""" 
    # tf.Graph().as_default() 的值返回 graph
    with tf.Graph().as_default() as graph:
      
        # 新建GraphDef文件，用于临时载入模型中的图 
        od_graph_def = tf.GraphDef()
        # 在空白图中加载GraphDef中的图，相当于把pb加载到了一个新定义的图中，所以需要新建一个tf.graphDef()
        
        # 接下來讀取pb模型文件(二進位)
        with tf.gfile.GFile(graph_file, 'rb') as fid:    
            #解析將二進位文件載入至od_graph_def
            od_graph_def.ParseFromString(fid.read())
            #將暫存的檔案(od_graph_def)傳回至tf.Graph().as_default() 也就是graph
            tf.import_graph_def(od_graph_def, name='')            
            
    return graph

        
def load_graph_2(graph_file):
    
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')          
            
    return graph

def load_name(graph_file):
    
    with tf.gfile.FastGFile(graph_file, 'rb') as f:
        frozen_graph_def = tf.GraphDef()
        frozen_graph_def.ParseFromString(f.read())
    graph = tf.Graph()
    return [n.name for n in graph.as_graph_def().node]
   

Below we load the graph and extract the relevant tensors using [`get_tensor_by_name`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name). These tensors reflect the input and outputs of the graph, or least the ones we care about for detecting objects.

In [16]:
detection_graph = load_graph(SSD_GRAPH_FILE)


# # The input placeholder for the image.
# # `get_tensor_by_name` returns the Tensor with the associated name in the Graph.
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# # Each box represents a part of the image where a particular object was detected.
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# # Each score represent how level of confidence for each of the objects.
# # Score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

# # The classification of the object (integer id).
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')


Run detection and classification on a sample image.

In [17]:
import os
IMAGE_DIR = "imgs"
IMAGE_PATHS = []

for file in os.listdir(IMAGE_DIR):
    if file.endswith(".jpeg"):
        IMAGE_PATHS.append(os.path.join(IMAGE_DIR, file))

IMAGE_PATHS

['imgs\\training_img-1.jpeg',
 'imgs\\training_img-263.jpeg',
 'imgs\\training_img-274.jpeg',
 'imgs\\training_img-97.jpeg']

In [18]:
# Load a sample image.
#image = Image.open('./assets/training_img-97.jpeg')
for image_path in IMAGE_PATHS:
    image = Image.open(image_path)
    # Turn PIL format to array for tensorflow
    image_np = np.expand_dims(np.asarray(image, dtype=np.uint8), 0)

    with tf.Session(graph=detection_graph) as sess:                
        # Actual detection.
        (boxes, scores, classes) = sess.run([detection_boxes, detection_scores, detection_classes], 
                                            feed_dict={image_tensor: image_np})

        # Remove unnecessary dimensions
        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes)
        #print(scores)
        #print(classes)
        confidence_cutoff = np.max(scores)
        # Filter boxes with a confidence score less than `confidence_cutoff`
        boxes, scores, classes = filter_boxes(confidence_cutoff, boxes, scores, classes)
        for i in range(len(boxes)):
            print("scores: ", scores[i], "class: ", classes[i])
        # The current box coordinates are normalized to a range between 0 and 1.
        # This converts the coordinates actual location on the image.
        width, height = image.size
        box_coords = to_image_coords(boxes, height, width)
        # Each class with be represented by a differently colored box
        draw_boxes(image, box_coords, classes)

        plt.figure(figsize=(12, 8))
        plt.imshow(image) 
    


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/Conv2D (defined at <ipython-input-15-8f165a749d3d>:61) ]]
	 [[Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Minimum_3/_127]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/Conv2D (defined at <ipython-input-15-8f165a749d3d>:61) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/Conv2D':
  File "C:\Users\ijan\anaconda3\envs\py36-tf114\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ijan\